# AirBnb NY Listing Price Prediction: ML Models Set up


Resources on distributing Python with Ray:
* Databricks [doc](https://docs.databricks.com/en/machine-learning/ray-integration.html)
* Ray [documentation](https://docs.ray.io/en/latest/ray-more-libs/joblib.html) for Scikit-learn


In [0]:
%pip install mlflow
%pip install scikit-learn==1.4.1.post1

In [0]:
import pandas as pd
import numpy as np
import mlflow
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

### Split into Train & Test

In [0]:
catalog_ = f"price_prediction"
schema_ = f"ny_listing"
spark.sql("USE CATALOG "+catalog_)
spark.sql("USE SCHEMA "+schema_)

In [0]:
SEED_ = 142

gold_data = spark.sql("SELECT * from gold_data")
train_df, test_df = gold_data.randomSplit([.85, .15], seed = SEED_)
train_df = train_df.toPandas()
test_df = test_df.toPandas()

display(train_df.head(2))
display(test_df.head(2))

In [0]:
# Convert 